In [49]:
import os 
import numpy as np 
import cv2
from glob import glob
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from albumentations import HorizontalFlip, CoarseDropout, RandomBrightnessContrast, RandomRotate90


In [50]:
def load_dataset(path):
    images = sorted(glob(os.path.jion(path, "images", "*")))
    masks = sorted(glob(os.path.jion(path, "masks", "*")))
    return images, masks

dataset_path = os.path.join("dataset2", "non-aug")

In [51]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [52]:
def conv_block(inputs, num_filters):
    x= Conv2D(num_filters, 3, padding="same")(inputs)
    x= BatchNormalization()(x)
    x=Activation('relu')(x)

    x=Conv2D(num_filters, 3, padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    return x

In [53]:
def save_dataset(images, masks, save_dir, augment=False):
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name= x.split("/")[-1].split(".")[0]
        x= cv2.imread(x, cv2.IMREAD_COLOR)
        y=cv2.imread(y,cv2.IMREAD_COLOR)
        if augment == True:
            aug= HorizontalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x1= augmented["image"]
            y1= augmented["mask"]

            aug= CoarseDropout(p=1, max_hole=10, max_height=8, max_width=8)
            augmented= aug(image=x, mask=y)
            x2=augmented['image']
            y2= augmented['mask']

            aug= RandomBrightnessContrast(p=1)
            augmented= aug(image=x, mask=y)
            x3=augmented['image']
            y3= augmented['mask']

            aug_x = [x, x1, x2, x3]
            aug_y = [y, y1, y2,y3]
        else:
            aug_x=[x]
            aug_y=[y]
        idx=0
        for ax , ay in zip(aug_x, aug_y):
            aug_name=f"{name}_{idx}.png"
            save_image_path= os.path.join(save_dir, "images", aug_name)
            save_mask_path = os.path.join(save_dir,"masks", aug_name)

            cv2.imwrite(save_image_path, ax)
            cv2.imwrite(save_mask_path, ay)

            idx +=1

In [54]:
save_dir = os.path.join("dataset2", "non-aug")
for item in ["train", "valid", "test"]:
    create_dir(os.path.join(save_dir, item, "images"))
    create_dir(os.path.join(save_dir, item, "masks"))

In [55]:
def load_data(path):
    train_x= sorted(glob(os.path.join(path, "train", "images", "*" )))
    train_y= sorted(glob(os.path.join(path, "train", "masks", "*" )))
    valid_x= sorted(glob(os.path.join(path, "valid", "images", "*" )))
    valid_y= sorted(glob(os.path.join(path, "valid", "masks", "*" )))

    return (train_x, train_y), (valid_x, valid_y)

In [56]:
(train_x, train_y), (valid_x, valid_y)= load_data(dataset_path)

print("train" , len(train_x), len(train_y))
print("valid" , len(valid_x), len(valid_y))

save_dataset(train_x, train_y, os.path.join(save_dir, "train"), augment=True)
save_dataset(valid_x, valid_y, os.path.join(save_dir, "valid"), augment=True)

train 239 239
valid 1542 1542


100%|██████████| 1542/1542 [00:05<00:00, 277.39it/s]
